In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
import pandas as pd 
import collections
import numpy as np 
import os
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [2]:
class vae_model(tf.keras.Model):
    def __init__(self,z_dim,input_dim,hidden_num=2):
        super(vae_model, self).__init__()
        self.input_dim=input_dim
        self.z_dim=z_dim
        self.hidden_num=hidden_num#选择几层编码层，1表示只用64,2表示用[64,128]，3表示只用128
        self.encoder_f1=tf.keras.layers.Dense(128,activation='relu')
        self.encoder_f2=tf.keras.layers.Dense(64,activation='relu')
        self.encoder_f3_mean=tf.keras.layers.Dense(self.z_dim)#求均值的层
        self.encoder_f3_var=tf.keras.layers.Dense(self.z_dim)#求var的层
        
        self.decoder_f1=tf.keras.layers.Dense(128,activation='relu')
        self.decoder_f2=tf.keras.layers.Dense(64,activation='relu')
        self.decoder_f3=tf.keras.layers.Dense(self.input_dim,activation='relu')

    # encoder传播的过程
    def encoder(self, x):
        if self.hidden_num==3:
            h=self.encoder_f1(x)
            mu=self.encoder_f3_mean(h)
            log_var=self.encoder_f3_var(h)
        elif self.hidden_num==2:
            h=self.encoder_f1(x)
            h=self.encoder_f2(h)
            mu=self.encoder_f3_mean(h)
            log_var=self.encoder_f3_var(h)
        else:
            h=self.encoder_f2(x)
            mu=self.encoder_f3_mean(h)
            log_var=self.encoder_f3_var(h)

        return  mu, log_var

    # decoder传播的过程
    def decoder(self, z):
        if self.hidden_num==3:
            h=self.decoder_f1(z)
            output=self.decoder_f3(h)
        elif self.hidden_num==2:
            h=self.decoder_f2(z)
            h=self.decoder_f1(h)
            output=self.decoder_f3(h)
        else:
            h=self.decoder_f2(z)
            output=self.decoder_f3(h)
        return output

    def reparameterize(self, mu, log_var):
        eps = tf.random.normal(log_var.shape)
        std = tf.exp(log_var)         # 去掉log, 得到方差；
        std = std**0.5                # 开根号，得到标准差；
        z = mu + std * eps
        return z

    def call(self, inputs,training=None):
        mu, log_var = self.encoder(inputs)
        z = self.reparameterize(mu, log_var)
        x_hat = self.decoder(z)
        return x_hat, mu, log_var

class VAE():
    def __init__(self,data_path,z_dim=2,max_epochs=1000,batch_size=128,patient_num=50,hidden_num=1,lr=1e-4,delta=1e-4):
#         super(VAE,self).__init__()
        self.data_path=data_path
        self.max_epochs=max_epochs
        self.batch_size=batch_size
        self.patient_num=patient_num
        self.z_dim=z_dim
        self.hidden_num=hidden_num
        self.lr=lr
        self.delta=delta#验证集误差低于该值时，迭代停止
        
        self.load_data()        
        self.input_dim=self.data_label_1_x.shape[1]
        
        #定义VAE的结构
        self.vae_model=vae_model(self.z_dim,self.input_dim,self.hidden_num)
#         self.vae_model.build(input_shape=(self.batch_size, self.input_dim))
        
    def load_data(self):
        """
        加载数据，得到少数类数据
        """
        self.data=pd.read_csv(self.data_path)
        self.label_num_dict=dict(collections.Counter(self.data['label']))
        self.label_0_num=self.label_num_dict[0]
        self.label_1_num=self.label_num_dict[1]
        self.need_add_label_num=self.label_0_num-self.label_1_num#需要增强的数据量
        self.data_label_0=self.data[self.data['label']==0]
        self.data_label_1=self.data[self.data['label']==1]
        
        self.data_label_1_x=self.data_label_1[self.data_label_1.columns[:-1]]
        self.data_label_1_y=self.data_label_1[self.data_label_1.columns[-1:]]
        
        #数据归一化处理
        self.scale_process()
        
        #数据打包
        self.train_x,self.train_y,self.train_dataset=self.zip_data(self.train_x,self.train_y)
        self.valid_x,self.valid_y,self.valid_dataset=self.zip_data(self.valid_x,self.valid_y)
        self.test_x,self.test_y,self.test_dataset=self.zip_data(self.test_x,self.test_y)

    def scale_process(self):
        """
        数据归一化处理
        数据切分
        """
        self.maxabs = preprocessing.MaxAbsScaler()
        self.data_label_1_x = self.maxabs.fit_transform(self.data_label_1_x)
        self.train_x,self.test_x,self.train_y,self.test_y=train_test_split(self.data_label_1_x,self.data_label_1_y,test_size=0.4)
        self.valid_x,self.test_x,self.valid_y,self.test_y=train_test_split(self.test_x,self.test_y,test_size=0.5)

    def zip_data(self,data_x,data_y):
        """
        以batch_size的大小将数据打包
        """
        train_data = tf.data.Dataset.from_tensor_slices(data_x).batch(self.batch_size)
        train_labels = tf.data.Dataset.from_tensor_slices(data_y.values[:,0]).batch(self.batch_size)
        train_dataset = tf.data.Dataset.zip((train_data,train_labels)).shuffle(True)
        return train_data,train_data,train_dataset
    
    def fit(self):
        """
        训练模型
        """
        global x_hat,x,mu, log_var,loss
        #创建模型参数
        self.vae_model.build(input_shape=(self.batch_size, self.input_dim))
        self.optimizer = tf.keras.optimizers.Adam(lr=self.lr)
        self.valid_loss_list=[]#记录每个epoch的验证集损失值，用于早停机制中
        self.smallest_loss=np.inf
        self.train_mu_list=[]
        self.train_logvar_list=[]
        
        for epoch in range(self.max_epochs):
            epoch_loss=0
            data_num=0
            for x,y in self.train_dataset:
                with tf.GradientTape() as tape:
                    x_hat, mu, log_var=self.vae_model(x)
                    rec_loss = tf.reduce_mean(tf.losses.MSE(x, x_hat))
                    kl_div = -0.5 * (log_var + 1 -mu**2 - tf.exp(log_var))
                    kl_div = tf.reduce_mean(kl_div) / x.shape[0]
                    
                    #误差求和
                    loss = rec_loss + 1. * kl_div
                
                grads = tape.gradient(loss, self.vae_model.trainable_variables)
                self.optimizer.apply_gradients(zip(grads, self.vae_model.trainable_variables))
                epoch_loss+=loss.numpy()*x.shape[0]
                data_num+=x.shape[0]
                
                self.train_mu_list+=mu.numpy().tolist()
                self.train_logvar_list+=log_var.numpy().tolist()
            
            
            #得到验证集的损失值，用于判断是否需要早停
            valid_loss=self.evaluate_loss()
            self.valid_loss_list+=[valid_loss]
            print(f'epoch={epoch},train_loss={epoch_loss/data_num},valid_loss={valid_loss}')
            
            if valid_loss<self.smallest_loss:
                self.best_vae_model=self.vae_model
                
            if self.patient_num!=None:
                #需要使用早停机制
                min_loss_index=np.argmin(self.valid_loss_list)#验证集损失值最小的epoch处
                if len(self.valid_loss_list)-min_loss_index>self.patient_num or valid_loss<self.delta:
                    #达到早停结束的时刻
                    break                
            
    def evaluate_loss(self):
        """
        计算验证集的损失值
        """
        epoch_loss=0
        data_num=0
        for x,y in self.valid_dataset:
            x_hat, mu, log_var=self.vae_model(x)
            rec_loss = tf.reduce_mean(tf.losses.MSE(x, x_hat))
            kl_div = -0.5 * (log_var + 1 -mu**2 - tf.exp(log_var))
            kl_div = tf.reduce_mean(kl_div) / x.shape[0]
            loss = rec_loss + 1. * kl_div
            
            epoch_loss+=loss.numpy()*x.shape[0]
            data_num+=x.shape[0]
        return epoch_loss/data_num
    
    def resample(self,creat_num=None):
        #生成z
        if creat_num==None or creat_num<=0:
            new_creat_z=np.random.normal(size=(self.need_add_label_num,self.z_dim))
        else:
            new_creat_z=np.random.normal(size=(creat_num,self.z_dim))
        #生成x
        new_creat_x_normal=self.best_vae_model.decoder(new_creat_z).numpy()
        #再反归一化
        new_creat_x=self.maxabs.inverse_transform(new_creat_x_normal)
        new_creat_x=pd.DataFrame(new_creat_x,columns=self.data_label_1.columns[:-1])
        new_creat_y=pd.DataFrame([1]*len(new_creat_x),columns=['label'])
        return new_creat_x,new_creat_y

In [4]:
for data_name in ['prosper(100)','gcdata']:
    print('creat data: ',data_name)
    data_path=f'./{data_name}.csv'
    vae=VAE(data_path,z_dim=4,max_epochs=100,batch_size=128,patient_num=5,hidden_num=1,lr=1e-4,delta=1e-4)
    vae.fit()
    new_sample_data_x,new_sample_data_y=vae.resample()
    new_sample_data=new_sample_data_x.T.append(new_sample_data_y.T).T
    if os.path.exists('./new_data/')==False:
        os.makedirs('./new_data/')
    new_sample_data.to_csv(f'./new_data/{data_name}.csv',encoding='utf_8_sig',index=False)

creat data:  prosper(100)
epoch=0,train_loss=0.19635505020618438,valid_loss=0.19708271324634552
epoch=1,train_loss=0.19713238060474395,valid_loss=0.19443483650684357
epoch=2,train_loss=0.19755476593971252,valid_loss=0.1971963793039322
epoch=3,train_loss=0.19325916945934296,valid_loss=0.19386297464370728
epoch=4,train_loss=0.1939113885164261,valid_loss=0.19560697674751282
epoch=5,train_loss=0.19600453197956086,valid_loss=0.19250980019569397
epoch=6,train_loss=0.19348806977272034,valid_loss=0.19267240166664124
epoch=7,train_loss=0.1926753932237625,valid_loss=0.18982240557670593
epoch=8,train_loss=0.19250185847282408,valid_loss=0.19221898913383484
epoch=9,train_loss=0.19230149686336517,valid_loss=0.19131238758563995
epoch=10,train_loss=0.18995550215244295,valid_loss=0.19014395773410797
epoch=11,train_loss=0.1892388367652893,valid_loss=0.188225656747818
epoch=12,train_loss=0.18805033802986146,valid_loss=0.1889350563287735
epoch=13,train_loss=0.18879729866981507,valid_loss=0.188131883740425